# Load Ntuple and DecayHash

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('/home/belle/zhangboy/inclusive_R_D/')
import utilities as util
import decayHash
from decayHash import DecayHashMap

Welcome to JupyROOT 6.24/06


In [2]:
import uproot
files = ['MC14ri_sigDDst_foldex_e_7/sigDDst.root:B0', 
         'MC14ri_normDDst_foldex_e_7/normDDst.root:B0',
         'MC14ri_Dststell2_foldex_e_8/Dststell2.root:B0', 
         'MC14ri_DststTau1_foldex_e_7/DststTau1.root:B0',
         'MC14ri_DststTau2_foldex_e_7/DststTau2.root:B0']

with uproot.open(f'../Samples/Signal_MC14ri/{files[2]}') as file:
    df_test = pd.DataFrame(file.arrays(entry_start=0,entry_stop=None,library="np")) # exclude bkg

In [3]:
df_test['B0_mcPDG'] = df_test['B0_mcPDG'].fillna(0)
df_test['D_mcPDG'] = df_test['D_mcPDG'].fillna(0)

In [6]:
df_test.columns

Index(['__experiment__', '__run__', '__event__', '__production__',
       '__candidate__', '__ncandidates__', '__MCDecayString__', '__weight__',
       'Ecms', 'ROEeidBDT',
       ...
       'e_genMotherPDG', 'e_mcErrors', 'e_mcPDG', 'e_dM', 'e_isBremsCorrected',
       'e_nPXDHits', 'e_isCloneTrack', 'e_d0_mcPDG', 'e_d1_mcPDG', 'e_pErr'],
      dtype='object', length=197)

In [4]:
df_test.eval(f'p_D_l = D_CMS_p + e_CMS_p', inplace=True)

In [96]:
df_test4 = df_test.query(f'abs(e_genMotherPDG)==521 and D_mcErrors<32 and e_mcErrors==2048 and B0_mcErrors==3384')[340:350]
df_test4[['B0_mcErrors','B0_mcPDG','D_mcErrors','D_mcPDG','e_genMotherPDG','e_mcErrors','e_mcPDG']]

,B0_mcErrors,B0_mcPDG,D_mcErrors,D_mcPDG,e_genMotherPDG,e_mcErrors,e_mcPDG
233084,3384.0,521.0,0.0,-411.0,521.0,2048.0,-11.0
234161,3384.0,-521.0,0.0,411.0,-521.0,2048.0,11.0
234419,3384.0,-521.0,0.0,411.0,-521.0,2048.0,11.0
235027,3384.0,521.0,0.0,-411.0,521.0,2048.0,-11.0
235369,3384.0,521.0,0.0,-411.0,521.0,2048.0,-11.0
237201,3384.0,521.0,0.0,-411.0,521.0,2048.0,-11.0
239418,3384.0,521.0,0.0,-411.0,521.0,2048.0,-11.0
239551,3384.0,-521.0,0.0,411.0,-521.0,2048.0,11.0
240253,3384.0,-521.0,0.0,411.0,-521.0,2048.0,11.0
241122,3384.0,-521.0,0.0,411.0,-521.0,2048.0,11.0


In [97]:
#df2=df_test.query('B0_mcErrors>=1176 and abs(B0_mcPDG)!=300553 and D_mcErrors==0 and e_mcErrors>=128 and e_genMotherPDG==B0_mcPDG and abs(e_mcPDG)==11')
candidate = df_test4.iloc[0]
#print(candidate['__MCDecayString__'])

In [6]:
decayhash=f'../Samples/Signal_MC14ri/MC14ri_Dststell2_foldex_e_8/hashmap_Dststell2.root'
hashmap1 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

In [98]:
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values
org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

300553
    -521
        423
            421
                130
                211
                -211
                111
                    22
                    22
            111
                22
                22
        11
        -12
    521
        -421
            211
            -211
            111
                22
                22
        111
            22
            22
        211
        -211
        211
        -211
        211



In [52]:
import ROOT
mode = '-521 (-> 413 -211 111 11 -12)'
decaytree = ROOT.Belle2.DecayTree(mode)
hashmap1.get_original_decay(candidate["B0_DecayHash"],candidate["B0_DecayHashEx"]).find_decay(decaytree)

True

In [21]:
def found(modes,row):
    for mode in modes:
        # check the decay chain for the reconstructed B meson only
        if mode.startswith(str(int(candidate['B0_mcPDG']))):
            print(mode)
            decaytree = ROOT.Belle2.DecayTree(mode)
            if hashmap1.get_original_decay(candidate["B0_DecayHash"],candidate["B0_DecayHashEx"]).find_decay(decaytree):
                return True
            else:
                continue
        else:
            continue
    return False

In [22]:
hash_modes = util.mode_dict['e']
for name,modes in hash_modes.items():
    if found(modes,candidate):
        print(util.DecayMode[name].value)
    else:
        print('No'+name)

Nosig_D_tau_nu
Nosig_D_l_nu
Nosig_Dst_tau_nu
Nosig_Dst_l_nu
NoDstst_tau_nu_mixed
-521 (-> 10423 15 (-> 11 -12 16) -16)
-521 (-> 10421 15 (-> 11 -12 16) -16)
-521 (-> 20423 15 (-> 11 -12 16) -16)
-521 (-> 425 15 (-> 11 -12 16) -16)
NoDstst_tau_nu_charged
Nores_Dstst_l_nu_mixed
-521 (-> 10423 13 -14)
-521 (-> 10421 13 -14)
-521 (-> 20423 13 -14)
-521 (-> 425 13 -14)
Nores_Dstst_l_nu_charged
Nononres_Dstst_l_nu_mixed
-521 (-> 411 211 13 -14)
-521 (-> 411 211 111 13 -14)
-521 (-> 413 211 13 -14)
-521 (-> 413 211 111 13 -14)
Nononres_Dstst_l_nu_charged
Nogap_Dstst_l_nu_mixed


In [20]:
candidate[['e_CMS_mcP','e_CMS_p','e_pErr']]

e_CMS_mcP    1.936044
e_CMS_p      2.428414
e_pErr       0.018288
Name: 480236, dtype: object

In [5]:
import uproot
cabinetry_signal = uproot.concatenate(['../Samples/prediction.root:signal'],library="pd")
cabinetry_bkg = uproot.concatenate(['../Samples/prediction.root:background'],library="pd")

In [6]:
cabinetry_signal.head()

,jet_pt,weight,lep_charge
0,625.623216,0.023243,1
1,595.394006,0.023483,-1
2,536.036954,0.024020,-1
3,528.128782,0.024891,-1
4,547.928949,0.025518,-1


In [18]:
signal = cabinetry_signal.query('lep_charge>0')
bkg = cabinetry_bkg.query('lep_charge>0')
signal_hist = np.histogram(signal['jet_pt'], bins=[200, 300, 400, 500, 600],weights=signal['weight'])
bkg_hist = np.histogram(bkg['jet_pt'], bins=[200, 300, 400, 500, 600],weights=bkg['weight'])
signal_err = np.histogram(signal['jet_pt'], bins=[200, 300, 400, 500, 600],weights=signal['weight']**2)
bkg_err = np.histogram(bkg['jet_pt'], bins=[200, 300, 400, 500, 600],weights=bkg['weight']**2)

In [20]:
np.sqrt(signal_err[0])

array([0.        , 0.19931166, 0.77410713, 0.78830503])

In [15]:
signal_hist

(array([ 0.        ,  1.58536913, 23.6164268 , 24.54892223]),
 array([200, 300, 400, 500, 600]))

In [4]:
# read a signal mc as a pandas dataframe
files = ['MC14ri_sigDDst_foldex_e_7/sigDDst_0.parquet', 
         'MC14ri_normDDst_foldex_e_7/normDDst_0.parquet',
         'MC14ri_Dststell2_foldex_e_7/Dststell2_0.parquet', 
         'MC14ri_DststTau1_foldex_e_7/DststTau1_0.parquet',
         'MC14ri_DststTau2_foldex_e_7/DststTau2_0.parquet']

# filename = f'../Samples/Signal_MC14ri/{files[2]}'
# decayhash = f'../Samples/Signal_MC14ri/MC14ri_Dststell2_foldex_e_7/hashmap_Dststell2.root'
filename=f'../Samples/Signal_MC_ROEx1/B2Dst_l_nu/Ntuples/MC_e_10k.parquet'
decayhash=f'../Samples/Signal_MC_ROEx1/B2Dst_l_nu/Ntuples/hashmap_MC_e_10k.root'
hashmap1 = DecayHashMap(decayhash, removeRadiativeGammaFlag=False)
#hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)
data = pd.read_parquet(filename, engine="pyarrow")
#data.eval('e_pSig = (e_CMS_mcP - e_CMS_p)/e_pErr', inplace=True)
df, samples=util.get_dataframe_samples(data, 'e', template=False)

for name in samples:
    print(f'{name}: {len(samples[name])}')

$D\tau\nu$: 0
$D^\ast\tau\nu$: 0
$D^{\ast\ast}\tau\nu$: 0
$D\ell\nu$: 5
$D^\ast\ell\nu$: 2636
$D^{\ast\ast}\ell\nu$: 3
bkg_fakeDTC: 625
bkg_fakeB: 252
bkg_continuum: 0
bkg_others: 0


In [7]:
samples[r'$D^{\ast\ast}\ell\nu$'][['B0_genNMissingDst','B0_genNMissingpi0','B0_genNMissinggamma']].value_counts()

B0_genNMissingDst  B0_genNMissingpi0  B0_genNMissinggamma
1                  1                  2                      1360
                                      3                       883
                                      4                       288
                                      5                        60
                                      1                        30
                                      6                        12
                   0                  1                         1
                                      2                         1
                   1                  7                         1
dtype: int64

In [8]:
samples[r'$D^{\ast\ast}\ell\nu$'][['B0_genNMissingDst','B0_genNMissingpi0','B0_genNMissinggamma']].value_counts()

B0_genNMissingDst  B0_genNMissingpi0  B0_genNMissinggamma
0                  1                  2                      1
1                  2                  4                      1
                                      6                      1
dtype: int64

In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

# e channel

In [23]:
pd.set_option('display.max_rows', None)
a=samples[r'$D^{\ast\ast}\ell\nu$'].query('B0_CMS3_weMissM2>8 and p_D_l<1')[['B0_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
a

B0_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
 521.0     521.0          0.0         1320.0       0.0         6            4
-521.0    -521.0          0.0         296.0        0.0         6            3
 521.0     521.0          0.0         312.0        0.0         6            2
                                      1336.0       0.0         6            2
-521.0    -521.0          0.0         312.0        0.0         6            1
dtype: int64

In [20]:
candidate = samples[r'$D^{\ast\ast}\ell\nu$'].query('B0_CMS3_weMissM2>8 and p_D_l<1').iloc[6]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'e_mcErrors','e_isBremsCorrected','B0_CMS3_weMissM2','e_pSig']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG                 511.0
D_mcPDG                 -411.0
e_mcPDG                  -11.0
e_genMotherPDG           511.0
DecayMode                    6
B0_mcErrors               24.0
D_mcErrors                 0.0
e_mcErrors                 0.0
e_isBremsCorrected         0.0
B0_CMS3_weMissM2      9.059644
e_pSig                3.630975
Name: 50688, dtype: object
300553
    511
        -10411
            -411
                321
                -211
                -211
            111
                22
                22
        -11
        12
    -511
        413
            421
                -10323
                    -323
                        -321
                        111
                            22
                            22
                    111
                        22
                        22
                -13
                14
            211
        -10431
            -431
                333
                    130
                    310
         

In [21]:
print(candidate['__MCDecayString__'])

Content of MCParticle list
╰── Upsilon(4S) (300553)
    ├── B0 (511)
    │   ├── D_0*- (-10411)
    │   │   ├── D- (-411)
    │   │   │   ├── K+ (321)
    │   │   │   │   ├╶╶ e- (11)
    │   │   │   │   ├╶╶ nu_mu (14)
    │   │   │   │   ╰╶╶ mu+ (-13)
    │   │   │   │       ├╶╶ anti-nu_mu (-14)
    │   │   │   │       ├╶╶ nu_e (12)
    │   │   │   │       ╰╶╶ e+ (-11)
    │   │   │   ├── pi- (-211)
    │   │   │   ╰── pi- (-211)
    │   │   ╰── pi0 (111)
    │   │       ├── gamma (22)
    │   │       ╰── gamma (22)
    │   ├── e+ (-11)
    │   ╰── nu_e (12)
    ╰── anti-B0 (-511)
        ├── D*+ (413)
        │   ├── D0 (421)
        │   │   ├── K_1- (-10323)
        │   │   │   ├── K*- (-323)
        │   │   │   │   ├── K- (-321)
        │   │   │   │   │   ├╶╶ gamma (22)
        │   │   │   │   │   ╰╶╶ gamma (22)
        │   │   │   │   ╰── pi0 (111)
        │   │   │   │       ├── gamma (22)
        │   │   │   │       ╰── gamma (22)
        │   │   │   ╰── pi0 (111)
        │   │ 

In [16]:
df3 = samples[r'$D\ell\nu$'].query('B0_CMS3_weMissM2>8 and p_D_l<1')
df3[['__event__','B0_mcErrors','e_mcErrors','B0_CMS_p','D_CMS_mcP','D_CMS_p','e_CMS_mcP','e_CMS_p','e_pSig','B0_CMS3_weMissM2','p_D_l']]

,__event__,B0_mcErrors,e_mcErrors,B0_CMS_p,D_CMS_mcP,D_CMS_p,e_CMS_mcP,e_CMS_p,e_pSig,B0_CMS3_weMissM2,p_D_l
38740,8198493,2056.0,2048.0,0.961089,1.328134,1.320325,1.583429,1.026613,56.063556,2.977534,2.346938
39404,1372477,2188.0,2180.0,1.450823,1.031608,1.032232,1.240519,0.826538,59.536626,3.322220,1.858770
55971,34434797,8.0,0.0,1.338455,1.148966,1.149381,1.154281,0.509834,362.513188,4.905236,1.659215
73278,5939869,8.0,0.0,0.993786,1.051094,1.050751,1.488461,0.708447,591.061317,4.959367,1.759197
119911,2459735,8.0,0.0,1.159548,1.288300,1.290189,1.322123,0.815593,190.065856,3.518582,2.105782
126545,8748384,2056.0,2048.0,1.654234,1.317880,1.281719,1.232314,0.738880,44.955691,2.584715,2.020599
179925,22575596,8.0,0.0,1.008443,1.528610,1.526374,1.937666,0.628086,877.360666,4.025019,2.154461
247727,37154896,8.0,0.0,1.140189,1.113648,1.112609,1.437532,1.100309,168.007385,2.758081,2.212918
251989,9089867,8.0,0.0,1.286374,1.656673,1.655751,1.908782,0.506599,481.262032,3.566608,2.162351
261857,18733482,8.0,0.0,1.477670,1.225286,1.222806,1.003199,0.521974,201.632287,4.241776,1.744780


In [14]:
pd.set_option('display.max_rows', None)
b=df.query('B0_mcErrors==24')[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
b

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         4            15939
-511.0     411.0    11.0    -511.0          0.0         24.0         0.0         4            15621
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         6             5844
-511.0     411.0    11.0    -511.0          0.0         24.0         0.0         6             5752
 511.0    -411.0   -11.0     431.0          0.0         24.0         0.0         0               40
-511.0     411.0    11.0     15.0           0.0         24.0         0.0         3               34
 511.0    -411.0   -11.0    -15.0           0.0         24.0         0.0         3               23
-511.0     411.0    11.0    -431.0          0.0         24.0         0.0         0               21
                             15.0           0.0         24.0         0.0         0               18
 511.0   

In [32]:
candidate = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2').iloc[4]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'e_mcErrors','e_isBremsCorrected','e_d0_mcPDG','e_d1_mcPDG','B0_CMS3_weMissM2']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG                -511.0
D_mcPDG                  411.0
e_mcPDG                   11.0
e_genMotherPDG          -511.0
DecayMode                    2
B0_mcErrors                8.0
D_mcErrors                 0.0
e_mcErrors                 0.0
e_isBremsCorrected         0.0
e_d0_mcPDG                11.0
e_d1_mcPDG                 NaN
B0_CMS3_weMissM2      3.879121
Name: 5468, dtype: object
300553
    511
        -411
            321
            -321
            -211
            111
                22
                22
        223
            -211
            211
            111
                22
                22
        323
            311
                310
            211
        -311
            310
    -511
        411
            -321
            211
            211
        11
        -12
        22



# Mu Channel

In [29]:
pd.set_option('display.max_rows', None)
a=samples[r'bkg_others'][['B0_mcPDG','D_mcPDG','mu_mcPDG','mu_genMotherPDG','D_mcErrors','B0_mcErrors','mu_mcErrors','DecayMode']].value_counts()
a

B0_mcPDG  D_mcPDG  mu_mcPDG  mu_genMotherPDG  D_mcErrors  B0_mcErrors  mu_mcErrors  DecayMode
-511.0     411.0    13.0     -511.0           0.0         140.0        2180.0       7            6
 511.0    -411.0   -13.0      511.0           0.0         136.0        2176.0       7            6
                                                          140.0        2180.0       7            6
-511.0     411.0    13.0     -511.0           0.0         136.0        2176.0       7            5
                                                                       128.0        7            1
dtype: int64

In [14]:
df3 = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2')
df3[['D_mcPDG','mu_mcPDG','mu_genMotherPDG','B0_mcErrors','D_mcErrors','mu_mcErrors','B0_CMS3_weMissM2','p_D_l']]

,D_mcPDG,mu_mcPDG,mu_genMotherPDG,B0_mcErrors,D_mcErrors,mu_mcErrors,B0_CMS3_weMissM2,p_D_l
5424,-411.0,-13.0,511.0,8.0,0.0,0.0,4.415084,1.488679
9489,-411.0,-13.0,511.0,8.0,0.0,0.0,3.340456,1.946793
11970,411.0,13.0,-511.0,8.0,0.0,0.0,4.258790,1.887789
13022,411.0,13.0,-511.0,8.0,0.0,0.0,3.854828,1.695532
13885,411.0,13.0,-511.0,8.0,0.0,0.0,4.319704,1.954827


In [28]:
df3 = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2')
df3[['D_mcPDG','mu_mcPDG','mu_genMotherPDG','B0_mcErrors','D_mcErrors','mu_mcErrors','B0_CMS3_weMissM2','p_D_l']]

,D_mcPDG,mu_mcPDG,mu_genMotherPDG,B0_mcErrors,D_mcErrors,mu_mcErrors,B0_CMS3_weMissM2,p_D_l
5350,-411.0,-13.0,511.0,8.0,0.0,0.0,4.415084,1.488679
9361,-411.0,-13.0,511.0,8.0,0.0,0.0,3.340456,1.946793
11810,411.0,13.0,-511.0,8.0,0.0,0.0,4.258790,1.887789
12843,411.0,13.0,-511.0,8.0,0.0,0.0,3.854828,1.695532
13694,411.0,13.0,-511.0,8.0,0.0,0.0,4.319704,1.954827


In [23]:
candidate = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2').iloc[4]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','mu_mcPDG','mu_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'mu_mcErrors','B0_CMS3_weMissM2']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG              -511.0
D_mcPDG                411.0
mu_mcPDG                13.0
mu_genMotherPDG       -511.0
DecayMode                  7
B0_mcErrors              8.0
D_mcErrors               0.0
mu_mcErrors              0.0
B0_CMS3_weMissM2    4.319704
Name: 13885, dtype: object
300553
    511
        -413
            -421
                -20213
                    113
                        211
                        -211
                    -211
                321
            -211
        223
            -211
            211
            111
                22
                22
        211
        -213
            -211
            111
                22
                22
        213
            211
            111
                22
                22
    -511
        411
            -321
            211
            211
        13
        -14
        22
        22

